En este archivo Phyton vamos a realizar la carga y tratamiento de los archivos proporcionados por la empresa Pontia World. El objetivo es tener los DF finales para analisis de KPI de negocio en SQL

# 💻ETL DATOS - EXTRACCION, LIMPIEZA Y CARGA DE JSON💻

#📚Importación de librerias Phyton y carga json

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from os import sep
import json
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving emocion.json to emocion.json
Saving id_visitante-atracciones.json to id_visitante-atracciones.json
Saving id_visitante-duracion.json to id_visitante-duracion.json
Saving id_visitante-procedencia.json to id_visitante-procedencia.json
Saving id_visitante-ticket.json to id_visitante-ticket.json
Saving valoraciones.json to valoraciones.json


# 🕵🏻 Exploración de **emocion.json**

In [3]:
ruta_json = '/content/emocion.json'

# Leer el JSON en un DataFrame
df = pd.read_json(ruta_json, encoding='utf-8')

# Mostrar el DataFrame para verificar
df.head(5)
df.shape

(3, 35885)

**Transformar el DataFrame**

Objetivo: Convertir las 35,885 columnas en filas, con columnas separadas para t_id, emocion, y tiempo_recogida.

Acción: Transponer el DataFrame con df.T para que cada fotografía sea una fila, y renombrar las columnas.

In [4]:
# Transponer para que columnas sean filas
df_emocion = df.T

# Renombrar columnas (opcional, para claridad)
df_emocion.columns = ['t_id', 'emocion', 'tiempo_recogida']

#devolvemos el index

df_emocion = df_emocion.reset_index(drop=True)

# Mostrar el resultado
df_emocion.head()

,t_id,emocion,tiempo_recogida
0,Training_10118481.jpg,angry,291
1,Training_10120469.jpg,angry,425
2,Training_10131352.jpg,angry,499
3,Training_10161559.jpg,angry,715
4,Training_1021836.jpg,angry,301


**Exploración inicial de los datos**

Objetivo:
1.   Traducir las emociones a castellado, ya que nuestra empresa es Española
2.   Entender y trabajar la columnda **tiempo_recogida** para sacar conclusiones y entender coherencia de los datos



In [5]:
# Exploración inicial
print("Frecuencia de emociones:")
print(df_emocion["emocion"].value_counts())

print("\nEstadísticas de tiempo_recogida:")
print(df_emocion["tiempo_recogida"].describe())

# Diccionario de traducción
traduccion_emociones = {
    'happy': 'feliz',
    'neutral': 'neutral',
    'sad': 'triste',
    'fear': 'miedo',
    'angry': 'enojado',
    'surprise': 'sorpresa',
    'disgust': 'asco'
}
# Cambio de idioma de las emociones
df_emocion['emocion'] = df_emocion['emocion'].replace(traduccion_emociones)
df_emocion.shape



Frecuencia de emociones:
emocion
happy       8901
neutral     6138
sad         6019
fear        5075
angry       4905
surprise    3953
disgust      543
Name: count, dtype: int64

Estadísticas de tiempo_recogida:
count     35885
unique      718
top         150
freq         68
Name: tiempo_recogida, dtype: int64


(35885, 3)

In [6]:
# Convertir a formato legible la fecha y hora
fecha_base = datetime(2022, 9, 1, 7, 0)
df_emocion['fecha_hora'] = df_emocion['tiempo_recogida'].apply(lambda x: fecha_base + timedelta(hours=int(x)))
df_emocion.head(5)

,t_id,emocion,tiempo_recogida,fecha_hora
0,Training_10118481.jpg,enojado,291,2022-09-13 10:00:00
1,Training_10120469.jpg,enojado,425,2022-09-19 00:00:00
2,Training_10131352.jpg,enojado,499,2022-09-22 02:00:00
3,Training_10161559.jpg,enojado,715,2022-10-01 02:00:00
4,Training_1021836.jpg,enojado,301,2022-09-13 20:00:00


In [7]:
max_tiempo_recogida = df_emocion['tiempo_recogida'].max()

print("=== Valor máximo de tiempo_recogida ===")
print("Máximo:", max_tiempo_recogida)

=== Valor máximo de tiempo_recogida ===
Máximo: 719


In [8]:
valores_unicos = np.sort(df_emocion['tiempo_recogida'].unique())

print("=== Valores únicos de tiempo_recogida (ordenados de menor a mayor) ===")
print(valores_unicos)
print("\nNúmero de valores únicos:", len(valores_unicos))

=== Valores únicos de tiempo_recogida (ordenados de menor a mayor) ===
[2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76
 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244
 245 246 247 248 249 250 251 252 253 254 255 256 25

In [9]:
moda_tiempo_recogida = df_emocion['tiempo_recogida'].mode()[0]
print("El valor que más se repite en tiempo_recogida es:", moda_tiempo_recogida)

El valor que más se repite en tiempo_recogida es: 150


In [10]:
# Horas faltantes en formato legible
horas_faltantes = set(range(717)) - set(df_emocion["tiempo_recogida"])
fechas_faltantes = [fecha_base + timedelta(hours=h) for h in horas_faltantes]
print("Horas faltantes en formato legible:")
for fecha in sorted(fechas_faltantes):
    print(fecha)

Horas faltantes en formato legible:
2022-09-01 07:00:00
2022-09-01 08:00:00


In [11]:
#creamos nuevas filas en el data frame con los duplicados, en base a la información que tenemos en las fotografías:
nuevas_filas = pd.DataFrame([
    {'t_id': 'Training_8475486_doble.jpg', 'emocion': 'enojado'},
    {'t_id': 'Training_52789098_doble.jpg', 'emocion': 'neutral'}
])

# Añadirlas al DataFrame original
df_emocion = pd.concat([df_emocion, nuevas_filas], ignore_index=True)

#comrpobamos numero de filas
print(df_emocion.shape)

(35887, 4)


In [12]:
#Rellenamos los nulos con emocion_desconocida para poder hacer una buena evalución posteriormente.
df_emocion['emocion'] = df_emocion['emocion'].fillna('emocion_desconocida')

#Rellenamos los nulos con la moda de la columna tiempo_recogida
df_emocion['fecha_hora'] = df_emocion['fecha_hora'].fillna("2022-09-07 13:00:00")

#Rellenamos los nulos con la moda de la columna tmpo_recogida
df_emocion['tiempo_recogida'] = df_emocion['tiempo_recogida'].fillna(150)

<ipython-input-12-75b42afb5ff3>:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_emocion['tiempo_recogida'] = df_emocion['tiempo_recogida'].fillna(150)


In [13]:
# Inspeccionar nulos después de la imputación
print("\n=== Inspección de nulos después de la imputación ===")
print("Filas totales:", len(df_emocion))
print("Nulos por columna:\n", df_emocion.isnull().sum())
df_emocion.shape



=== Inspección de nulos después de la imputación ===
Filas totales: 35887
Nulos por columna:
 t_id               0
emocion            0
tiempo_recogida    0
fecha_hora         0
dtype: int64


(35887, 4)

In [14]:
#en vista a los duplicados que hemos visto, vamos a comprobar que emocion tienen asociada. Vemos que efectivamente, coincide con lo que vimos en las
#fotos que tenemos en el data base
print(df_emocion[df_emocion['t_id'].str.contains('8475486')])
print(df_emocion[df_emocion['t_id'].str.contains('52789098')])

                             t_id   emocion  tiempo_recogida  \
3303         Training_8475486.jpg  sorpresa              432   
35885  Training_8475486_doble.jpg   enojado              150   

               fecha_hora  
3303  2022-09-19 07:00:00  
35885 2022-09-07 13:00:00  
                              t_id   emocion  tiempo_recogida  \
18129        Training_52789098.jpg  sorpresa              386   
35886  Training_52789098_doble.jpg   neutral              150   

               fecha_hora  
18129 2022-09-17 09:00:00  
35886 2022-09-07 13:00:00  


In [ ]:
df_emocion.shape

(35887, 4)

#👀 Exploración de valoraciones.json

In [15]:
# Importo el json "valoraciones" localizado en la carpeta compartida de PROYECTO JÚPITER

import json

ruta_json = '/content/valoraciones.json'

# Abrir y cargar el archivo JSON
with open(ruta_json, 'r', encoding='utf-8') as archivo:
    datos = json.load(archivo)

# Mostrar contenido del JSON - vemos que la clase es un diccionario
print(datos)
print(type(datos))

{'Training_10118481.jpg': 2, 'Training_10120469.jpg': 6, 'Training_10131352.jpg': 0, 'Training_10161559.jpg': 4, 'Training_1021836.jpg': 6, 'Training_10269675.jpg': 3, 'Training_10278738.jpg': 6, 'Training_10290703.jpg': 0, 'Training_10295477.jpg': 1, 'Training_10315441.jpg': 4, 'Training_10316849.jpg': 0, 'Training_10333072.jpg': 7, 'Training_10334355.jpg': 2, 'Training_10345473.jpg': 1, 'Training_10422050.jpg': 5, 'Training_10485618.jpg': 9, 'Training_10503476.jpg': 1, 'Training_10524198.jpg': 3, 'Training_10539399.jpg': 2, 'Training_10592361.jpg': 3, 'Training_10595751.jpg': 8, 'Training_10608067.jpg': 0, 'Training_10740356.jpg': 5, 'Training_10851653.jpg': 3, 'Training_10857340.jpg': 2, 'Training_10882484.jpg': 5, 'Training_10899258.jpg': 2, 'Training_10922970.jpg': 3, 'Training_10967257.jpg': 5, 'Training_11023881.jpg': 1, 'Training_11036720.jpg': 6, 'Training_11102431.jpg': 0, 'Training_11175213.jpg': 10, 'Training_11185740.jpg': 1, 'Training_1119091.jpg': 8, 'Training_11206889.j

In [16]:
datos_valoraciones = [] #creamos una lista vacia para almacenar los datos

for archivo, valoracion in datos.items(): #queremos que en el bucle for para cada archivo nos guarde el diccionario en la variable valoracion
    datos_valoraciones.append({
        't_id': archivo,
        'valoracion': valoracion
    })

# Verificar resultado
print(datos_valoraciones[:5])  # Muestra las primeras 5 entradas

df_valoraciones = pd.DataFrame(datos_valoraciones)

print(df_valoraciones.shape)
print(df_valoraciones.describe())
df_valoraciones.columns
df_valoraciones.head()

[{'t_id': 'Training_10118481.jpg', 'valoracion': 2}, {'t_id': 'Training_10120469.jpg', 'valoracion': 6}, {'t_id': 'Training_10131352.jpg', 'valoracion': 0}, {'t_id': 'Training_10161559.jpg', 'valoracion': 4}, {'t_id': 'Training_1021836.jpg', 'valoracion': 6}]
(35885, 2)
         valoracion
count  35885.000000
mean       4.983921
std        3.153761
min        0.000000
25%        2.000000
50%        5.000000
75%        8.000000
max       10.000000


,t_id,valoracion
0,Training_10118481.jpg,2
1,Training_10120469.jpg,6
2,Training_10131352.jpg,0
3,Training_10161559.jpg,4
4,Training_1021836.jpg,6


In [17]:
#Comprobamos que no hay nulos
df_valoraciones.isnull().sum()

,0
t_id,0
valoracion,0


In [18]:
#observamos que la valoración es un object pero vemos que es un entero, asi que cambiamos el tipo de dato
#y lo pasamos a entero, comprobando depues
df_valoraciones['valoracion'] = df_valoraciones['valoracion'].astype(int)
df_valoraciones.dtypes
print(df_valoraciones.shape)

(35885, 2)


In [19]:
#creamos nuevas filas en el data frame con los duplicados. De momento no añadiremos
#la info de la valoración ya que lo haremos en base a la media de la emoción en el merge de ambos df
nuevas_filas = pd.DataFrame([
    {'t_id': 'Training_8475486_doble.jpg'},
    {'t_id': 'Training_52789098_doble.jpg'}
])

# Añadirlas al DataFrame original
df_valoraciones = pd.concat([df_valoraciones, nuevas_filas], ignore_index=True)

#comrpobamos numero de filas
print(df_valoraciones.shape)

(35887, 2)


In [20]:
print(df_valoraciones[df_valoraciones['t_id'].str.contains('8475486')])
print(df_valoraciones[df_valoraciones['t_id'].str.contains('52789098')])

                             t_id  valoracion
3303         Training_8475486.jpg         9.0
35885  Training_8475486_doble.jpg         NaN
                              t_id  valoracion
18129        Training_52789098.jpg         5.0
35886  Training_52789098_doble.jpg         NaN


# MERGE DE EMOCION Y VALORACIONES

In [25]:
emocion_valoraciones = pd.merge(df_emocion, df_valoraciones, on='t_id', how='inner') #how='inner':
# solo conservará las filas que tienen el mismo t_id en ambos

#comprobamos numero de filas y head
print(emocion_valoraciones.shape)
emocion_valoraciones.head()
emocion_valoraciones.tail()

(35887, 5)


,t_id,emocion,tiempo_recogida,fecha_hora,valoracion
35882,PublicTest_98972870.jpg,sorpresa,255,2022-09-11 22:00:00,10.0
35883,PublicTest_99242645.jpg,sorpresa,573,2022-09-25 04:00:00,5.0
35884,PublicTest_99446963.jpg,sorpresa,466,2022-09-20 17:00:00,0.0
35885,Training_8475486_doble.jpg,enojado,150,2022-09-07 13:00:00,NaN
35886,Training_52789098_doble.jpg,neutral,150,2022-09-07 13:00:00,NaN


In [26]:
#calculamos la valoracion media de la emocion ¨enojado¨ y ¨neutral¨
media_enojado = emocion_valoraciones[emocion_valoraciones['emocion'] == 'enojado']['valoracion'].mean()
print(f"Valoración media para 'enojado': {media_enojado}")

media_neutral = emocion_valoraciones[emocion_valoraciones['emocion'] == 'neutral']['valoracion'].mean()
print(f"Valoración media para 'neutral': {media_neutral}")

Valoración media para 'enojado': 4.971049949031601
Valoración media para 'neutral': 4.930922124470512


In [27]:
emocion_valoraciones.tail()

,t_id,emocion,tiempo_recogida,fecha_hora,valoracion
35882,PublicTest_98972870.jpg,sorpresa,255,2022-09-11 22:00:00,10.0
35883,PublicTest_99242645.jpg,sorpresa,573,2022-09-25 04:00:00,5.0
35884,PublicTest_99446963.jpg,sorpresa,466,2022-09-20 17:00:00,0.0
35885,Training_8475486_doble.jpg,enojado,150,2022-09-07 13:00:00,NaN
35886,Training_52789098_doble.jpg,neutral,150,2022-09-07 13:00:00,NaN


In [28]:
#Insertamos la media de las emociones (5 en este caso) para los campos que tenemos nulos:
emocion_valoraciones['valoracion'] = emocion_valoraciones['valoracion'].fillna(5)

In [29]:
emocion_valoraciones.tail()

,t_id,emocion,tiempo_recogida,fecha_hora,valoracion
35882,PublicTest_98972870.jpg,sorpresa,255,2022-09-11 22:00:00,10.0
35883,PublicTest_99242645.jpg,sorpresa,573,2022-09-25 04:00:00,5.0
35884,PublicTest_99446963.jpg,sorpresa,466,2022-09-20 17:00:00,0.0
35885,Training_8475486_doble.jpg,enojado,150,2022-09-07 13:00:00,5.0
35886,Training_52789098_doble.jpg,neutral,150,2022-09-07 13:00:00,5.0


In [30]:
#Convertimos la valoración a INT
emocion_valoraciones['valoracion'] = emocion_valoraciones['valoracion'].astype(int)
emocion_valoraciones.head(2)

,t_id,emocion,tiempo_recogida,fecha_hora,valoracion
0,Training_10118481.jpg,enojado,291,2022-09-13 10:00:00,2
1,Training_10120469.jpg,enojado,425,2022-09-19 00:00:00,6


In [31]:
emocion_valoraciones.isnull().sum()

,0
t_id,0
emocion,0
tiempo_recogida,0
fecha_hora,0
valoracion,0


In [32]:
emocion_valoraciones = emocion_valoraciones.drop(columns=['tiempo_recogida'])
emocion_valoraciones.tail()


,t_id,emocion,fecha_hora,valoracion
35882,PublicTest_98972870.jpg,sorpresa,2022-09-11 22:00:00,10
35883,PublicTest_99242645.jpg,sorpresa,2022-09-25 04:00:00,5
35884,PublicTest_99446963.jpg,sorpresa,2022-09-20 17:00:00,0
35885,Training_8475486_doble.jpg,enojado,2022-09-07 13:00:00,5
35886,Training_52789098_doble.jpg,neutral,2022-09-07 13:00:00,5


In [35]:
ruta_base = "/content/"
nombre_archivo = "emocion_valoraciones"
ruta_completa = f"{ruta_base}{nombre_archivo}.csv"

emocion_valoraciones.to_csv(
    ruta_completa,
    index=False,
    encoding="utf-8",   # cambiar a Latin si se necesitan los caracteres especiales
    sep=",",            # más estándar
    na_rep="NULL"
)
print(f"Archivo guardado: {ruta_completa}")

Archivo guardado: /content/emocion_valoraciones.csv


#🌏 Exploración de id_visitante_procedencia.json

In [38]:
path_to_file = "/content/id_visitante-procedencia.json"

# leer archivo json
with open(path_to_file, 'r') as file:
    procedencia= json.load(file)
print(procedencia)

Output hidden; open in https://colab.research.google.com to view.

In [39]:
#creamos una lista vacia para almacenar los datos
datos_procedencia = []

for id_visitante, lista_id in procedencia.items():
    for elemento in lista_id:  # cada elemento es un diccionario individual
        elemento["id_visitante"] = int(id_visitante)
        datos_procedencia.append(elemento)

df_procedencia = pd.DataFrame(datos_procedencia)

df_procedencia.dtypes
df_procedencia.describe()
df_procedencia[df_procedencia.duplicated(subset='t_id', keep=False)] #estos son los registros duplicados de la columna t_id




,t_id,procedencia,id_visitante
2484,Training_8475486.jpg,España,74
15577,Training_52789098.jpg,España,469
16312,Training_8475486.jpg,España,495
21163,Training_52789098.jpg,India,675


In [40]:
#en base a lo que hemos hecho en emociones que es añadir las filas de fotos que faltaban, renombramos las fotos duplicadas con el sufijo_doble
#para poder mantener los registros

# Detectar duplicados completos (ambos)
duplicado = df_procedencia.duplicated('t_id', keep=False)

# Identificar primera ocurrencia de cada duplicado en el DataFrame completo
primera_ocurrencia = df_procedencia.duplicated('t_id', keep='last')

# Aplicar cambio solo a esas primeras ocurrencias
df_procedencia.loc[primera_ocurrencia, 't_id'] = (
    df_procedencia.loc[primera_ocurrencia, 't_id']
    .str.replace('.jpg', '_doble.jpg', regex=False)
)

#comrpobar que lo hemos hecho bien
print(df_procedencia[df_procedencia['t_id'].str.contains('8475486')])

                             t_id procedencia  id_visitante
2484   Training_8475486_doble.jpg      España            74
16312        Training_8475486.jpg      España           495


In [41]:
df_procedencia.head() #podemos observar que un unico id_visitante tiene varias procedencia, analizaremos mas porque no parece correcto

,t_id,procedencia,id_visitante
0,Training_10118481.jpg,Filipinas,3
1,Training_15361067.jpg,España,3
2,Training_16232328.jpg,Puerto Rico,3
3,Training_27705571.jpg,Paraguay,3
4,Training_63766171.jpg,Argentina,3


Consideramos que el id_visitante es erróneo, ya que agrupa 1 visitante con varias procedencia, cuando esta deberia ser una caracteristica unica, por lo que no vamos a modificar el df en base a esta columna, sino que nos centraremos en el t_id.

En EDA exploraremos igualmente sobre las procedencias por visitante, pero para la exportación, lo dejaremos así.

#🧭Exploración de id_visitante_duracion.json

In [42]:
# Como el json es complejo lo vamos a leer tal cual es, como un diccionario

#leer json de manera estandard - hemos encontrado ese trozo de codigo en bibliografia

with open("/content/id_visitante-duracion.json", "r", encoding="utf-8") as file:
    datos = json.load(file)

# Ver estructura y efectivamente es un diccionario
print(type(datos))

#creamos una lista vacia para almacenar los datos
datos_duracion =[]

#a las primeras claves, los numeros que contienen las listas dentro , la llamaremos ¨id_visitante¨
#porque es el numero que corresponde a cada visitante
#hacemos un doble for (for dentro de for) - primero recorremos las claves numericas y luego las listas dento de cada clave
for id_visitante , lista in datos.items(): #recorremos cada diccionario con clave numerica : primera vuelta 3, segunda 4 etc
  for elemento_listas in lista: #recorremos cada diccionario dentro de esa lista
     elemento_listas["id_visitante"] = int(id_visitante)
     datos_duracion.append(elemento_listas)

df_duracion=pd.DataFrame(datos_duracion)
df_duracion.head(20)

print(df_duracion.shape)
df_duracion.describe()


<class 'dict'>
(35887, 3)


,duracion,id_visitante
count,35887.000000,35887.000000
mean,359.748934,625.675760
std,100.122517,433.719407
min,-39.000000,1.000000
25%,292.000000,260.000000
50%,359.000000,553.000000
75%,428.000000,937.000000
max,811.000000,1787.000000


In [43]:
#convertimos duracion a int
df_duracion["duracion"] = df_duracion["duracion"].astype(int)
df_duracion["duracion"].describe()
df_duracion[df_duracion.duplicated(subset='t_id', keep=False)] #estos son los registros duplicados de la columna t_id

,t_id,duracion,id_visitante
2484,Training_8475486.jpg,346,74
15577,Training_52789098.jpg,336,469
16312,Training_8475486.jpg,507,495
21163,Training_52789098.jpg,160,675


In [44]:
#en base a lo que hemos hecho en emociones que es añadir las filas de fotos que faltaban, renombramos las fotos duplicadas con el sufijo_doble
#para poder mantener los registros

# Detectar duplicados completos (ambos)
duplicado = df_duracion.duplicated('t_id', keep=False)

# Identificar primera ocurrencia de cada duplicado en el DataFrame completo
primera_ocurrencia = df_duracion.duplicated('t_id', keep='last')

# Aplicar cambio solo a esas primeras ocurrencias
df_duracion.loc[primera_ocurrencia, 't_id'] = (
    df_duracion.loc[primera_ocurrencia, 't_id']
    .str.replace('.jpg', '_doble.jpg', regex=False)
)

#comrpobar que lo hemos hecho bien
print(df_duracion[df_duracion['t_id'].str.contains('8475486')])
print(df_duracion[df_duracion['t_id'].str.contains('52789098')])

                             t_id  duracion  id_visitante
2484   Training_8475486_doble.jpg       346            74
16312        Training_8475486.jpg       507           495
                              t_id  duracion  id_visitante
15577  Training_52789098_doble.jpg       336           469
21163        Training_52789098.jpg       160           675


In [45]:
#Calculamos la media de duracion en el df_duracion
media_duracion = df_duracion['duracion'].mean()
print(f"La media de la duración es {media_duracion.round(0)}")

La media de la duración es 360.0


Observamos de lo anterior que con la variable duracion nos ocurre algo parecido que con procedencia. Procedemos entonces a aplicar el mismo tratamiento, pero en vez de hacer la moda, creemos conveniente elegir la media de duraciones, comprobando primero que ninguna duracion supera las 9 horas

In [46]:
#comprobamos valores negativos en duracion (tiempo no puede ser negativo)
df_negativos = df_duracion[df_duracion['duracion'] < 0 & df_duracion['id_visitante']]
df_negativos

,t_id,duracion,id_visitante
493,Training_10422050.jpg,-39,17


In [47]:
#Como solo es un valor negativo, y no sabemos muy bien por qué, optamos por sustituirlo por un valor positivo considerandolo un error de imputacion

# Imputar la media en valores negativos de la columna coste
df_duracion['duracion'] = df_duracion['duracion'].abs()
print(df_duracion['duracion'].describe())  # Resumen estadístico

count    35887.000000
mean       359.751108
std        100.114707
min          1.000000
25%        292.000000
50%        359.000000
75%        428.000000
max        811.000000
Name: duracion, dtype: float64


Distribución relativamente simétrica:
Media (359.75) ≈ Mediana (359.0), lo que indica que la distribución no está sesgada ni a la izquierda ni a la derecha.El valor mínimo es 1. ¿Tiene sentido que una visita dure 1 minuto? Tendremos que analizar. La desviación estándar es 100: bastante grande. Aproximadamente un 68% de las duraciones estarán entre 260 y 460 (media ± desviación estándar).

In [ ]:
#comprobamos de nuevo que no hay valores negativos en duracion:
df_duracion[df_duracion['duracion'] < 0 & df_duracion['id_visitante']]

,t_id,duracion,id_visitante


In [48]:
df_duracion['duracion'] =df_duracion['duracion'].astype(int)
df_duracion.head(2)

,t_id,duracion,id_visitante
0,Training_10118481.jpg,308,3
1,Training_15361067.jpg,344,3


In [49]:
df_duracion.shape

(35887, 3)

#🎢Exploración de id_visitante_atracciones.json

In [69]:
#Como no puedo abrirlo con pandas,leer json de manera estandard.
with open('/content/id_visitante-atracciones.json', 'r', encoding="utf-8") as file:
    data = json.load(file)
#Confirmamos que lo que tenemos es un diccionario
type(data)

dict

In [71]:
#Para convertirlo en dataframe, vamos a hacer un bucle. Hacemos un primer bucle para las claves, y luego un segundo para las listas dentro del valor del diccionario.
datos_atracciones = []

for id_visitante , lista in data.items():
  for elemento_listas in lista:
     elemento_listas["id_visitante"] = int(id_visitante)
     datos_atracciones.append(elemento_listas)
df_atracciones=pd.DataFrame(datos_atracciones)


#Vamos a cambiar la columna 'comienzo atraccion' por enteros

# Rellenar con un valor por defecto, por ejemplo 0
df_atracciones['comienzo_atraccion'] = df_atracciones['comienzo_atraccion'].astype('Int64')


#Vamos a convertir el 'comienzo_atraccion' en un formato legible.
#Ponemos como fecha base el 1 de septiembre de 2022 a las 7:00h.

#pd.notnull(x) → verifica que el valor no sea NaN

#pd.NaT → es el equivalente nulo para fechas en Pandas
df_atracciones.loc[df_atracciones['comienzo_atraccion'] < 0, 'comienzo_atraccion'] *= -1

fecha_base = datetime(2022, 9, 1, 7, 0)

df_atracciones['comienzo_atraccion_fecha_hora'] = df_atracciones['comienzo_atraccion'].apply(
    lambda x: fecha_base + timedelta(hours=int(x)) if pd.notnull(x) else pd.NaT
)

df_atracciones.describe()
df_atracciones[df_atracciones.duplicated(subset='t_id', keep=False)] #estos son los registros duplicados

#como vemos, coinciden con los del archivo de duraciones (IMPORTANTE)



,t_id,atraccion,comienzo_atraccion,tiempo_de_espera,id_visitante,comienzo_atraccion_fecha_hora
2484,Training_8475486.jpg,Carrera de Autos Locos,529,11,74,2022-09-23 08:00:00
15577,Training_52789098.jpg,Aventuras Acuáticas,478,16,469,2022-09-21 05:00:00
16312,Training_8475486.jpg,Araña Saltarina,432,7,495,2022-09-19 07:00:00
21163,Training_52789098.jpg,Carros Chocones Divertidos,386,12,675,2022-09-17 09:00:00


In [72]:
#en base a lo que hemos hecho en emociones que es añadir las filas de fotos que faltaban, renombramos las fotos duplicadas con el sufijo_doble
#para poder mantener los registros

# Detectar duplicados completos (ambos)
duplicado = df_atracciones.duplicated('t_id', keep=False)

# Identificar primeras ocurrencia de cada duplicado en el DataFrame completo
primera_ocurrencia = df_atracciones.duplicated('t_id', keep='last')

# Aplicar cambio solo a esas segundas ocurrencias
df_atracciones.loc[primera_ocurrencia, 't_id'] = (
    df_atracciones.loc[primera_ocurrencia, 't_id']
    .str.replace('.jpg', '_doble.jpg', regex=False)
)

#comrpobar que lo hemos hecho bien
print(df_atracciones[df_atracciones['t_id'].str.contains('8475486')])
print(df_atracciones[df_atracciones['t_id'].str.contains('52789098')])

                             t_id               atraccion  comienzo_atraccion  \
2484   Training_8475486_doble.jpg  Carrera de Autos Locos                 529   
16312        Training_8475486.jpg         Araña Saltarina                 432   

       tiempo_de_espera  id_visitante comienzo_atraccion_fecha_hora  
2484                 11            74           2022-09-23 08:00:00  
16312                 7           495           2022-09-19 07:00:00  
                              t_id                   atraccion  \
15577  Training_52789098_doble.jpg         Aventuras Acuáticas   
21163        Training_52789098.jpg  Carros Chocones Divertidos   

       comienzo_atraccion  tiempo_de_espera  id_visitante  \
15577                 478                16           469   
21163                 386                12           675   

      comienzo_atraccion_fecha_hora  
15577           2022-09-21 05:00:00  
21163           2022-09-17 09:00:00  


In [73]:
df_atracciones.shape

(35887, 6)

In [74]:
#Vamos a calcular la moda del df_atraccion sin filtrar
moda_comienzo_atraccion = df_atracciones['comienzo_atraccion'].mode()[0]
print(f"La moda del comienzo de atraccion: {moda_comienzo_atraccion:.2f}")
moda_tiempo_espera = df_atracciones['tiempo_de_espera'].mode()[0]
print(f"La moda del tiempo de espera es: {moda_comienzo_atraccion:.2f}")

La moda del comienzo de atraccion: 1.00
La moda del tiempo de espera es: 1.00


In [75]:
#Hacemos la moda de todos los valores menos los negativos
df_atracciones_2 = df_atracciones[df_atracciones['comienzo_atraccion'] >= 0]
moda_comienzo_atraccion = df_atracciones_2['comienzo_atraccion'].mode()[0]
print(f"La moda del tiempo de espera es sin contar los valores negativos es: {moda_comienzo_atraccion:.2f}")

La moda del tiempo de espera es sin contar los valores negativos es: 1.00


In [76]:
# Buscamos la cantidad de nulos dentro de nuestro DF
print("\nNulos por columna:\n", df_atracciones.isnull().sum())
# podemos observar que en la columna "atracción " hay 1132 nulos en una columna que es importante para el desarrollo de nuestro análisis


Nulos por columna:
 t_id                                0
atraccion                        1138
comienzo_atraccion                 96
tiempo_de_espera                    0
id_visitante                        0
comienzo_atraccion_fecha_hora      96
dtype: int64


In [77]:
df_atracciones['comienzo_atraccion'].unique()

<IntegerArray>
[291, 180, 384, 625, 613, 533, 297,  82, 316, 552,
 ...
 468,   7, 153, 532, 708, 663, 149, 501, 198, 407]
Length: 720, dtype: Int64

In [78]:
#Imputaremos los datos nulos de la columna atracciones a atracciones desconocidas
df_atracciones['atraccion'] = df_atracciones['atraccion'].fillna('DESCONOCIDO')

#Imputaremos los datos nulos de comienzo de atracción a 150 ya que es la moda
df_atracciones['comienzo_atraccion'] = df_atracciones['comienzo_atraccion'].fillna(150)

#Imputaremos a 150 los datos negativos de dentro de la ambas columnas (comienzo_atraccion y comienzo_atraccion_fecha_hora) de tiempo ya que carecen de sentido porque al indicar
#en la columna "comienzo atraccion" el valor -1 sugiere un dato fuera del rango de 0 a 719 de nros enteros correspondientes
#a las horas, tomando en cuenta que el guíon dice específicamente que la hora de comienzo es 07:00 del 1 de septiembre de 2022 (hora 0 en nros enteros)
df_atracciones['comienzo_atraccion_fecha_hora'] = df_atracciones['comienzo_atraccion_fecha_hora'].fillna('2022-09-07 13:00:00')
df_atracciones.isnull().sum()

,0
t_id,0
atraccion,0
comienzo_atraccion,0
tiempo_de_espera,0
id_visitante,0
comienzo_atraccion_fecha_hora,0


In [79]:
print(df_atracciones['tiempo_de_espera'].describe())
print(df_atracciones['comienzo_atraccion'].describe())  # Resumen estadístico

count    35887.000000
mean        12.498036
std          4.014144
min         -3.000000
25%         10.000000
50%         12.000000
75%         15.000000
max         28.000000
Name: tiempo_de_espera, dtype: float64
count       35887.0
mean     357.872962
std      207.858678
min             1.0
25%           178.0
50%           357.0
75%           537.0
max           719.0
Name: comienzo_atraccion, dtype: Float64


In [80]:
valores_negativos = df_atracciones[df_atracciones['tiempo_de_espera'] < 0]
print(valores_negativos)

                           t_id                     atraccion  \
1817      Training_96216570.jpg               Selva Encantada   
2024      Training_50471741.jpg         Simulador Espacial 3D   
3014      Training_63106367.jpg         Caravana de Aventuras   
5590      Training_91511544.jpg           Torbellino Espacial   
5874      Training_61297982.jpg               Tren del Terror   
13817     Training_53475521.jpg               Selva Encantada   
14894     Training_91159308.jpg                Dragón Volador   
17978     Training_34669152.jpg            Carrusel Encantado   
19626     Training_57988945.jpg           Cine 4D Emocionante   
20913     Training_63023938.jpg        Carrera de Autos Locos   
23242     Training_94808846.jpg           Laberinto de Sueños   
24090     Training_44730165.jpg         Tobogán del Arco Iris   
24370     Training_76985291.jpg  Viaje al Centro de la Tierra   
28807  PrivateTest_24655069.jpg             Rápido del Trueno   
30958  PrivateTest_463311

In [81]:
#hacemos positivos los negativos
df_atracciones.loc[df_atracciones['tiempo_de_espera'] < 0, 'tiempo_de_espera'] *= -1
df_atracciones.loc[df_atracciones['comienzo_atraccion'] < 0, 'comienzo_atraccion'] *= -1

In [82]:
print(df_atracciones['tiempo_de_espera'].describe())  # Resumen estadístico

count    35887.000000
mean        12.499262
std          4.010324
min          0.000000
25%         10.000000
50%         12.000000
75%         15.000000
max         28.000000
Name: tiempo_de_espera, dtype: float64


#🎫Exploración de id_visitante_ticket.json

In [83]:
#Como no puedo abrirlo con pandas, abro el archivo de esta forma para poder verlo.
#Confirmamos que lo que tenemos es un diccionario

with open('/content/id_visitante-ticket.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

type(data)

dict

In [84]:
#Para convertirlo en dataframe, vamos a hacer un bucle. Hacemos un primer bucle para las claves, y luego un segundo para las listas dentro del valor del diccionario.

datos = []

for id, tablas in data.items():
  for tabla in tablas:
    tabla['id'] = int(id)
    datos.append (tabla)


In [85]:
#Lo convertimos en df
df_ticket = pd.DataFrame(datos)
df_ticket.info()
#Renombramos la columna para que coincida con los datos.
df_ticket.rename(columns = {'id':'id_visitante'}, inplace=True)
df_ticket.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   t_id                  35887 non-null  object 
 1   tipo_entrada          35887 non-null  object 
 2   coste                 35887 non-null  float64
 3   antelacion_de_compra  35887 non-null  int64  
 4   id                    35887 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.4+ MB


,t_id,tipo_entrada,coste,antelacion_de_compra,id_visitante
0,Training_10118481.jpg,Pase Anual,14.003708,0,3
1,Training_15361067.jpg,Entrada Familiar,20.900902,2,3
2,Training_16232328.jpg,Entrada Infantil,12.638039,218,3
3,Training_27705571.jpg,Paquete VIP,18.491775,39,3
4,Training_63766171.jpg,Entrada Familiar,18.233458,135,3


In [86]:
#redondeamos el coste a 2 decimales
df_ticket['coste'] = df_ticket['coste'].apply(lambda x: round(x, 2))
df_ticket.head()

,t_id,tipo_entrada,coste,antelacion_de_compra,id_visitante
0,Training_10118481.jpg,Pase Anual,14.00,0,3
1,Training_15361067.jpg,Entrada Familiar,20.90,2,3
2,Training_16232328.jpg,Entrada Infantil,12.64,218,3
3,Training_27705571.jpg,Paquete VIP,18.49,39,3
4,Training_63766171.jpg,Entrada Familiar,18.23,135,3


In [87]:
df_ticket[df_ticket.duplicated(subset='t_id', keep=False)] #estos son los registros duplicados

#como vemos, coinciden con los del archivo de duraciones, atracciones, procedencia (IMPORTANTE)

,t_id,tipo_entrada,coste,antelacion_de_compra,id_visitante
2484,Training_8475486.jpg,Pase Anual,14.16,230,74
15577,Training_52789098.jpg,Entrada Infantil,14.76,13,469
16312,Training_8475486.jpg,Entrada Individual,18.74,0,495
21163,Training_52789098.jpg,Paquete VIP,21.15,0,675


In [88]:
#en base a lo que hemos hecho en emociones que es añadir las filas de fotos que faltaban, renombramos las fotos duplicadas con el sufijo_doble
#para poder mantener los registros. Lo hacemos para la primera ocurrencia, ya que hemos visto que una fe las fotos que no aparecen corresponde a un bebe
#entonces, corresponderia a una entrada infantil

# Detectar duplicados completos (ambos)
duplicado = df_ticket.duplicated('t_id', keep=False)

# Identificar primera ocurrencia de cada duplicado en el DataFrame completo
primera_ocurrencia = df_ticket.duplicated('t_id', keep='last')

# Aplicar cambio solo a esass primera ocurrencias
df_ticket.loc[primera_ocurrencia, 't_id'] = (
    df_ticket.loc[primera_ocurrencia, 't_id']
    .str.replace('.jpg', '_doble.jpg', regex=False)
)

#comrpobar que lo hemos hecho bien
print(df_ticket[df_ticket['t_id'].str.contains('8475486')])
print(df_ticket[df_ticket['t_id'].str.contains('52789098')])


                             t_id        tipo_entrada  coste  \
2484   Training_8475486_doble.jpg          Pase Anual  14.16   
16312        Training_8475486.jpg  Entrada Individual  18.74   

       antelacion_de_compra  id_visitante  
2484                    230            74  
16312                     0           495  
                              t_id      tipo_entrada  coste  \
15577  Training_52789098_doble.jpg  Entrada Infantil  14.76   
21163        Training_52789098.jpg       Paquete VIP  21.15   

       antelacion_de_compra  id_visitante  
15577                    13           469  
21163                     0           675  


In [89]:
#Comprobamos que hay valores negativos:
coste_negativo = df_ticket[df_ticket['coste'] <= 0].shape[0]
print(f"Hay {coste_negativo} registros que tienen un coste negativo")

antelacion_negativo = df_ticket[df_ticket['antelacion_de_compra'] < 0].shape[0]
print(f"Hay {antelacion_negativo} registros que tienen una antelación de compra negativo")

id_negativo = df_ticket[df_ticket['id_visitante'] < 0].shape[0]
print(f"Hay {id_negativo} registros que tienen un id negativo")

Hay 13 registros que tienen un coste negativo
Hay 0 registros que tienen una antelación de compra negativo
Hay 0 registros que tienen un id negativo


Vemos que estos costes son mínimos (no superan los 3€) por lo que vamos a pasarlos a 0 para que por lo menos tengan un valor positivo.

No consideramos que tenga sentido pasar el valor a positivo ya que no parecen los costes reales de las entradas comparadas con el resto del df_ticket*.

*La comparativa de los precios por entreda está añadida en el EDA.

In [ ]:
# Imputar 0 en valores negativos de la columna coste
df_ticket.loc[df_ticket['coste'] < 0, 'coste'] = 0
print(df_ticket['coste'].describe())  # Resumen estadístico
print(df_ticket[df_ticket['coste'] < 0])  # Filas con valores negativos

count    35887.000000
mean        16.982984
std          4.974064
min          0.000000
25%         13.610000
50%         17.000000
75%         20.340000
max         39.330000
Name: coste, dtype: float64
Empty DataFrame
Columns: [t_id, tipo_entrada, coste, antelacion_de_compra, id_visitante]
Index: []


In [90]:
df_ticket.select_dtypes(include=['object', 'string']).columns

Index(['t_id', 'tipo_entrada'], dtype='object')

In [91]:
df_ticket['tipo_entrada'] = df_ticket['tipo_entrada'].str.lower() #pasamos a minuscula para evitar inconsistencias
df_ticket.head()

,t_id,tipo_entrada,coste,antelacion_de_compra,id_visitante
0,Training_10118481.jpg,pase anual,14.00,0,3
1,Training_15361067.jpg,entrada familiar,20.90,2,3
2,Training_16232328.jpg,entrada infantil,12.64,218,3
3,Training_27705571.jpg,paquete vip,18.49,39,3
4,Training_63766171.jpg,entrada familiar,18.23,135,3


In [ ]:
df_ticket.shape

(35887, 5)

# Merge de DF con ID VISITANTE

In [116]:
# Paso 1: merge de procedencia + ticket
df_1 = df_procedencia.merge(df_ticket.drop(columns=['id_visitante'], errors='ignore'), on='t_id', how='inner')

# Paso 2: merge del resultado anterior con atracciones
df_2 = df_1.merge(df_atracciones.drop(columns=['id_visitante'], errors='ignore'), on='t_id', how='inner')

# Paso 3: merge final con duración
df_id_visitante = df_2.merge(df_duracion.drop(columns=['id_visitante'], errors='ignore'), on='t_id', how='inner')

print(df_1.shape)
print(df_2.shape)
print(df_id_visitante.shape)
print(df_id_visitante.head())

(35887, 6)
(35887, 10)
(35887, 11)
                    t_id  procedencia  id_visitante      tipo_entrada  coste  \
0  Training_10118481.jpg    Filipinas             3        pase anual  14.00   
1  Training_15361067.jpg       España             3  entrada familiar  20.90   
2  Training_16232328.jpg  Puerto Rico             3  entrada infantil  12.64   
3  Training_27705571.jpg     Paraguay             3       paquete vip  18.49   
4  Training_63766171.jpg    Argentina             3  entrada familiar  18.23   

   antelacion_de_compra                atraccion  comienzo_atraccion  \
0                     0  Montaña Rusa de la Luna                 291   
1                     2        Mansión Embrujada                 180   
2                   218    Simulador Espacial 3D                 384   
3                    39     Fiesta de los Dulces                 625   
4                   135         Gran Caída Libre                 613   

   tiempo_de_espera comienzo_atraccion_fecha_hora  

Como hemos visto ID VISITANTE no es correcto en cuanto a identificador de persona unica (ya que las fotos son diferentes, la procedencia es diferente etc) asi que realmente no identifica nada correctamente como deberia. Por esa razon, entendemos que deberiamos, con los datos que tenemos, crear un ID de visita unica. Y decimos visita porque para identificar la persona como tal no tenemos mas que la procedencia, y no nos serviria, necesitariamos algo mas. Asi que, como digo, pensamos en crear una estimacion de visita unica utilizando 3 datos: procedencia, duracion, dia visita. Procedencia porquees el único dato constante y identificativo, duracion  porque distingue a diferentes estancias  por dia, y dia_visita porque separa días distintos. Creemos que este enfoque nos permite identificar correctamente las visitas al parque y a su vez evitar agrupar visitas diferentes bajo el mismo ID. con esta combinacion podriamos identificar de manera efectiva cada visita al parque aunque no tengamos un identificador persistente de persona.

In [117]:
#hacemos un drop de ID visitante y lo crearemos de nuevo
df_id_visitante = df_id_visitante.drop(columns='id_visitante')
df_id_visitante.head()

,t_id,procedencia,tipo_entrada,coste,antelacion_de_compra,atraccion,comienzo_atraccion,tiempo_de_espera,comienzo_atraccion_fecha_hora,duracion
0,Training_10118481.jpg,Filipinas,pase anual,14.00,0,Montaña Rusa de la Luna,291,5,2022-09-13 10:00:00,308
1,Training_15361067.jpg,España,entrada familiar,20.90,2,Mansión Embrujada,180,11,2022-09-08 19:00:00,344
2,Training_16232328.jpg,Puerto Rico,entrada infantil,12.64,218,Simulador Espacial 3D,384,19,2022-09-17 07:00:00,161
3,Training_27705571.jpg,Paraguay,paquete vip,18.49,39,Fiesta de los Dulces,625,9,2022-09-27 08:00:00,260
4,Training_63766171.jpg,Argentina,entrada familiar,18.23,135,Gran Caída Libre,613,17,2022-09-26 20:00:00,169


In [119]:

# Extraer solo la fecha del día de visita
df_id_visitante['dia_visita'] = df_id_visitante['comienzo_atraccion_fecha_hora'].dt.date

# Crear el ID numérico que agrupa por visita real en un día
df_id_visitante['id_visitante'] = df_id_visitante.groupby(['procedencia', 'duracion', 'dia_visita']).ngroup()+1
df_id_visitante.head()

,t_id,procedencia,tipo_entrada,coste,antelacion_de_compra,atraccion,comienzo_atraccion,tiempo_de_espera,comienzo_atraccion_fecha_hora,duracion,dia_visita,id_visitante
0,Training_10118481.jpg,Filipinas,pase anual,14.00,0,Montaña Rusa de la Luna,291,5,2022-09-13 10:00:00,308,2022-09-13,16656
1,Training_15361067.jpg,España,entrada familiar,20.90,2,Mansión Embrujada,180,11,2022-09-08 19:00:00,344,2022-09-08,12164
2,Training_16232328.jpg,Puerto Rico,entrada infantil,12.64,218,Simulador Espacial 3D,384,19,2022-09-17 07:00:00,161,2022-09-17,28210
3,Training_27705571.jpg,Paraguay,paquete vip,18.49,39,Fiesta de los Dulces,625,9,2022-09-27 08:00:00,260,2022-09-27,26769
4,Training_63766171.jpg,Argentina,entrada familiar,18.23,135,Gran Caída Libre,613,17,2022-09-26 20:00:00,169,2022-09-26,794


In [121]:
dia_visita = df_id_visitante['fecha'] = df_id_visitante['comienzo_atraccion_fecha_hora'].dt.date

visitantes_unicos_por_dia = df_id_visitante.groupby('dia_visita')['id_visitante'].nunique()
sum(visitantes_unicos_por_dia)

32137

In [122]:
#pasar a minuscula las columnas de texto
df_id_visitante['procedencia'] = df_id_visitante['procedencia'].str.lower() #pasamos a minuscula para evitar inconsistencias
df_id_visitante['atraccion'] = df_id_visitante['atraccion'].str.lower() #pasamos a minuscula para evitar inconsistencias
df_id_visitante.head()

,t_id,procedencia,tipo_entrada,coste,antelacion_de_compra,atraccion,comienzo_atraccion,tiempo_de_espera,comienzo_atraccion_fecha_hora,duracion,dia_visita,id_visitante,fecha
0,Training_10118481.jpg,filipinas,pase anual,14.00,0,montaña rusa de la luna,291,5,2022-09-13 10:00:00,308,2022-09-13,16656,2022-09-13
1,Training_15361067.jpg,españa,entrada familiar,20.90,2,mansión embrujada,180,11,2022-09-08 19:00:00,344,2022-09-08,12164,2022-09-08
2,Training_16232328.jpg,puerto rico,entrada infantil,12.64,218,simulador espacial 3d,384,19,2022-09-17 07:00:00,161,2022-09-17,28210,2022-09-17
3,Training_27705571.jpg,paraguay,paquete vip,18.49,39,fiesta de los dulces,625,9,2022-09-27 08:00:00,260,2022-09-27,26769,2022-09-27
4,Training_63766171.jpg,argentina,entrada familiar,18.23,135,gran caída libre,613,17,2022-09-26 20:00:00,169,2022-09-26,794,2022-09-26


# COMPROBANDO LAS REGLAS DE NEGOCIO

## La estancia de un visitante no puede superar las 9 horas ❌ no se cumple

In [123]:
#convertimos el comienzo de atraccion a datetime y extraemos fecha
df_id_visitante['fecha'] = df_id_visitante['comienzo_atraccion_fecha_hora'].dt.date

In [124]:
#Agrupé por id_visitante y fecha y sumé la duración total por visitante y por día:

duracion_por_dia = df_id_visitante.groupby(['id_visitante', 'fecha'])['duracion'].sum().reset_index()

In [127]:
duraciones_mayores = duracion_por_dia[duracion_por_dia['duracion'] > 540]
duraciones_mayores

,id_visitante,fecha,duracion
157,158,2022-09-20,554
182,183,2022-09-06,578
220,221,2022-09-29,602
226,227,2022-09-03,606
230,231,2022-09-17,606
...,...,...,...
32132,32133,2022-09-30,606
32133,32134,2022-09-15,638
32134,32135,2022-09-17,639
32135,32136,2022-09-05,657


He hecho un pequeño estudio de la duracion. Creo que debemos inputar los registros que no cumplen con la regla de negocio con un valor de 350 minutos. 350 minutos es muy representativo, lo he estudiado quitando los regsitros y sigue siendo la media cercana a 351-El valor 350 minutos está muy cerca de la media y la mediana de la distribución válida (mean ≈ 351.66, median ≈ 355.28).Esto lo convierte en un valor neutral y no sesgado: al imputar con él, no estamos empujando artificialmente los datos hacia extremos altos o bajos. De esta manera, tenemos datos relevantes y se evita la pérdida de datos para hacer agregaciones por dia/procedecnia/atracciones ya que eliminar esos registros puede afectar la calidad de tus KPIs si son datos reales que fallaron por error técnico o puntual. imputar nos permite conservar la estructura y volumen de datos a la vez que lo hemos estudiado y diremos que esos registros no cumplen la regla del negocio

In [128]:
media_valida = duracion_por_dia[duracion_por_dia['duracion'] <= 540]['duracion'].mean()
media_valida

np.float64(350.9902252443689)

In [129]:
df_id_visitante.loc[df_id_visitante['duracion'] > 540, 'duracion'] = 350
print(df_id_visitante['duracion'].describe())

count    35887.000000
mean       351.532923
std         90.408262
min          1.000000
25%        292.000000
50%        350.000000
75%        416.000000
max        540.000000
Name: duracion, dtype: float64


In [130]:
df_id_visitante['duracion'] = df_id_visitante['duracion'].astype(int)

##  Entradas fast pass se tienen que comprar con maximo 3 dias de antelacion ❌ no se cumple

In [134]:
#Comprobamos que las entradas de pase rápido se venden como máximo con 3 días de antelación
fast_pass = df_id_visitante[df_id_visitante['tipo_entrada'] == 'pase rápido']

contador = (fast_pass['antelacion_de_compra'] > 3).sum()

print(f"Hay {contador} entradas de 'Pase Rápido' vendidas con más de 3 días de antelación, que supone el {(contador*100)/35887:.2f}% de todas las entradas ")
print(f"Esto supone que un {(contador*100)/5986:.2f}% de las entradas de Pase Rápido se hayan vendido con más de 3 días de antelación")

Hay 3780 entradas de 'Pase Rápido' vendidas con más de 3 días de antelación, que supone el 10.53% de todas las entradas 
Esto supone que un 63.15% de las entradas de Pase Rápido se hayan vendido con más de 3 días de antelación


In [135]:
#vamos a ver una estadistica basica de las en
resumen = fast_pass['antelacion_de_compra'].describe()
print(resumen)

count    5986.000000
mean      116.481290
std       121.541217
min         0.000000
25%         0.000000
50%        78.000000
75%       222.000000
max       364.000000
Name: antelacion_de_compra, dtype: float64


In [137]:
#Vamos a ver estadistica basica de las violaciones de las entradas fast pass

# Filtrar violaciones: entradas "Pase Rápido" con más de 3 días de antelación
violaciones_rapido = df_id_visitante[
    (df_id_visitante['tipo_entrada'] == 'pase rápido') &
    (df_id_visitante['antelacion_de_compra'] > 3)
]

# Resumen estadístico de la antelación de compra en esas violaciones
resumen_violaciones = violaciones_rapido['antelacion_de_compra'].describe()
print("Resumen de antelación para entradas 'Pase Rápido' que violan la regla (> 3 días):")
print(resumen_violaciones)

Resumen de antelación para entradas 'Pase Rápido' que violan la regla (> 3 días):
count    3780.000000
mean      184.226720
std       104.587329
min         4.000000
25%        93.000000
50%       184.500000
75%       277.000000
max       364.000000
Name: antelacion_de_compra, dtype: float64


Consideraremos que es un error de etiquetado y reetiquetamos las entrads > de 3 dias para mantener los registros.

In [143]:
df_id_visitante.loc[
    (df_id_visitante['tipo_entrada'] == 'pase rápido') &
    (df_id_visitante['antelacion_de_compra'] > 3),
    'tipo_entrada'
] = 'pase rapido erroneo'

In [144]:
(df_id_visitante['tipo_entrada'] == 'pase rapido erroneo').sum() #comprobar cuántos fueron corregidos

np.int64(3780)

## No se pueden subir mas de 500 visitas a la misma atraccion en una misma hora ✅ se cumple

In [145]:
# Crear columna con la hora redondeada
df_id_visitante['hora'] = df_id_visitante['comienzo_atraccion_fecha_hora'].dt.floor('h')

#contar visitantes por atraccion y hora
conteo_por_hora = df_id_visitante.groupby(['atraccion', 'hora']).size().reset_index(name='visitantes')

#filtrar violaciones que incumplan la regla impuesta
violaciones_max500 = conteo_por_hora[conteo_por_hora['visitantes'] > 500]
violaciones_max500

,atraccion,hora,visitantes


In [146]:
media_visitantes_por_atraccion = conteo_por_hora.groupby('atraccion')['visitantes'].mean().reset_index()
media_visitantes_por_atraccion = media_visitantes_por_atraccion.sort_values(by='visitantes', ascending=False)
print(media_visitantes_por_atraccion)

                       atraccion  visitantes
13                   desconocido    2.007055
1            aventuras acuáticas    1.894249
29              tirolina extrema    1.883459
15         espejos de la risueña    1.876448
34                  vuelo mágico    1.867784
10             circus fantástico    1.863884
20             mansión embrujada    1.862637
18           jardín de las hadas    1.860075
0                araña saltarina    1.859813
16          fiesta de los dulces    1.859779
35    vuelta al mundo en 80 días    1.858929
23       mundo de las maravillas    1.857934
2        barco pirata misterioso    1.853659
11            cohetes galácticos    1.852830
12                cúpula estelar    1.850746
31           torbellino espacial    1.850662
4         carrera de autos locos    1.848881
9                 circuito veloz    1.846154
30         tobogán del arco iris    1.844156
28         simulador espacial 3d    1.841808
25             rápido del trueno    1.836066
17        

In [147]:
#queremos ver el promedio diario por atraccion, a ver si se acerca al limite impuesto por el negocio

# Contar visitantes por atracción y día
visitantes_por_dia = df_id_visitante.groupby(['atraccion', 'fecha']).size().reset_index(name='visitantes')

# Calcular el promedio diario por atracción
promedio_diario = visitantes_por_dia.groupby('atraccion')['visitantes'].mean().reset_index()
promedio_diario = promedio_diario.sort_values(by='visitantes', ascending=False)

# Mostrar las 10 atracciones con mayor promedio diario
print(promedio_diario.head(10))

                     atraccion  visitantes
13                 desconocido   36.709677
35  vuelta al mundo en 80 días   33.580645
10           circus fantástico   33.129032
1          aventuras acuáticas   32.935484
20           mansión embrujada   32.806452
22     montaña rusa de la luna   32.774194
24         rueda de la fortuna   32.709677
5   carros chocones divertidos   32.580645
25           rápido del trueno   32.516129
16        fiesta de los dulces   32.516129


In [148]:
#Agrupar por atracción y contar cuántos visitantes totales tiene cada una y ordenad de mayor a menor
demanda_total = df_id_visitante.groupby('atraccion').size().reset_index(name='total_visitantes')
demanda_total = demanda_total.sort_values(by='total_visitantes', ascending=False)
demanda_total.head(10)

,atraccion,total_visitantes
13,desconocido,1138
35,vuelta al mundo en 80 días,1041
10,circus fantástico,1027
1,aventuras acuáticas,1021
20,mansión embrujada,1017
22,montaña rusa de la luna,1016
24,rueda de la fortuna,1014
5,carros chocones divertidos,1010
25,rápido del trueno,1008
16,fiesta de los dulces,1008


In [ ]:
#creamos una funcion de resumen general que usaremos para entender un poco nuestros resultados
#y si tenemos que aplicar algun tipo mas de tratamiento
def resumen_general(df):
    resumen = pd.DataFrame({
        "Columnas": df.columns,
        "Tipo de Dato": df.dtypes,
         "Valores No Nulos": df.count(),
        "Valores Únicos": df.nunique(),
        "Valores Nulos": df.isnull().sum(),
        "Porcentaje Nulos (%)": df.isnull().mean() * 100,
        "Valores Duplicados": df.apply(lambda col: col.duplicated().sum()),
    })
    return resumen

resumen_valoraciones = resumen_general(df_valoraciones)
resumen_emocion = resumen_general(df_emocion)
resumen_duracion = resumen_general(df_duracion)
resumen_atracciones = resumen_general(df_atracciones)
resumen_procedencia = resumen_general (df_procedencia)
resumen_ticket = resumen_general (df_ticket)
resumen_emocion_valoraciones = resumen_general(emocion_valoraciones)
resumen_id_visitante = resumen_general(df_id_visitante)


resumen_total = pd.concat({
    "valoraciones": resumen_valoraciones,
    "emocion": resumen_emocion,
    "duracion": resumen_duracion,
    "atracciones": resumen_atracciones,
    "procedencia": resumen_procedencia,
    "ticket": resumen_ticket,
 "Merge emovion y valoraciones": resumen_emocion_valoraciones,
    "Merge DF con ID VISITANTE": resumen_id_visitante
})
resumen_total

# EDAs

# 🕵 EDA EMOCIÓN

In [149]:
# Los nulos que se presentan en las filas de emociones hemos decidido imputarlos a emociones desconocidas

# Calcular el número de filas con y sin nulos en la columna emocion

nulos = df_emocion['emocion'].isnull().sum()
no_nulos = len(df_emocion) - nulos

# Crear DataFrame para el gráfico
df_nulos = pd.DataFrame({
    'Estado': ['Datos Válidos', 'Datos Nulos'],
    'Cantidad': [no_nulos, nulos]
})

# Calcular porcentajes para el texto
df_nulos['Porcentaje'] = (df_nulos['Cantidad'] / len(df_emocion) * 100).round(2)

# Crear gráfico de barras
fig = px.bar(df_nulos, x='Estado', y='Cantidad',
             title='Comparación de Datos Válidos vs. Datos Nulos en la Columna Emoción',
             labels={'Cantidad': 'Número de Filas', 'Estado': 'Estado de los Datos'},
             color='Estado',
             text='Porcentaje',
             color_discrete_map={'Datos Válidos': '#00CC96', 'Datos Nulos': '#EF553B'})

# Personalizar el gráfico
fig.update_traces(texttemplate='%{text:.2f}%', textposition='auto')
fig.update_layout(yaxis_title='Número de Filas',
                  xaxis_title='Estado de los Datos',
                  showlegend=True,
                  bargap=0.3)

# Mostrar el gráfico
fig.show()

In [150]:
# Heatmap de las distribuciones de las emociones en el parque
fig2 = px.density_heatmap(df_emocion, x="fecha_hora", y="emocion",
                          marginal_x="histogram", marginal_y="histogram",
                          title="Distribución de emociones en el Parque (Septiembre 2022)",
                          range_x=["2022-09-01 00:00:00", "2022-10-01 06:00:00"],
                          nbinsx=100)

# Personalización
fig2.update_layout(yaxis_title='Emociones',
                   xaxis_title='Fecha y Hora',
                   showlegend=False,
                   bargap=0.2)

# Mostrar el heatmap
fig2.show()

Output hidden; open in https://colab.research.google.com to view.

In [151]:
# Calcular los porcentajes de cada emoción
emociones_counts = df_emocion['emocion'].value_counts(normalize=True) * 100
df_porcentajes = emociones_counts.reset_index()
df_porcentajes.columns = ['Emoción', 'Porcentaje']

# Crear gráfico de barras
fig = px.bar(df_porcentajes, x='Emoción', y='Porcentaje',
             title='Media de Porcentajes de Emociones en el Parque',
             labels={'Porcentaje': 'Porcentaje (%)', 'Emoción': 'Emoción'},
             color='Emoción',
             text='Porcentaje')

# Personalizar el gráfico
fig.update_traces(texttemplate='%{text:.1f}%', textposition='auto')
fig.update_layout(yaxis_title='Porcentaje (%)',
                  xaxis_title='Emoción',
                  showlegend=False,
                  bargap=0.2)

# Mostrar el gráfico
fig.show()

In [152]:
# Después de conocer los porcentajes de las emociones decidimos agrupar las emociones en 2 grupos: emociones positivas y emociones negativas;
# de esta forma podemos tener una visión general de las emociones que han sentido los usuarios durante su visita en el parque

# Definir categorías de emociones
positivas = ['feliz', 'sorpresa']
negativas = ['triste', 'miedo', 'enojado', 'asco']
neutral = ['neutral']

# Calcular porcentajes de cada emoción
emociones_counts = df_emocion['emocion'].value_counts(normalize=True) * 100
df_porcentajes = emociones_counts.reset_index()
df_porcentajes.columns = ['Emoción', 'Porcentaje']

# Agrupar por categorías
positivas_porcentaje = df_porcentajes[df_porcentajes['Emoción'].isin(positivas)]['Porcentaje'].sum()
negativas_porcentaje = df_porcentajes[df_porcentajes['Emoción'].isin(negativas)]['Porcentaje'].sum()
neutral_porcentaje = df_porcentajes[df_porcentajes['Emoción'].isin(neutral)]['Porcentaje'].sum()

# Crear DataFrame para el gráfico
df_categorias = pd.DataFrame({
    'Categoría': ['Positivas', 'Negativas', 'Neutral'],
    'Porcentaje': [positivas_porcentaje, negativas_porcentaje, neutral_porcentaje]
})

# Crear gráfico de barras
fig = px.bar(df_categorias, x='Categoría', y='Porcentaje',
             title='Porcentaje de Emociones Positivas, Negativas y Neutrales en el Parque',
             labels={'Porcentaje': 'Porcentaje (%)', 'Categoría': 'Categoría de Emoción'},
             color='Categoría',
             text='Porcentaje',
             color_discrete_map={'Positivas': '#00CC96', 'Negativas': '#EF553B', 'Neutral': '#636EFA'})

# Personalizar el gráfico
fig.update_traces(texttemplate='%{text:.1f}%', textposition='auto')
fig.update_layout(yaxis_title='Porcentaje (%)',
                  xaxis_title='Categoría de Emoción',
                  showlegend=True,
                  bargap=0.3)

# Mostrar el gráfico
fig.show()

#👀 EDA VALORACIONES

In [153]:
# En esta ocasión no usamos el df_valoraciones exclusivamente, sino que usamos en MERGE para tener todos los datos exactos
# Comprobar la presencia de valoraciones negativas - El resultado es FALSE, luego no hay ninguna valoración negativa
print((emocion_valoraciones['valoracion'] < 0).any())

False


In [154]:
# Media, moda y mediana
media = emocion_valoraciones['valoracion'].mean()
mediana = emocion_valoraciones['valoracion'].median()
moda = emocion_valoraciones['valoracion'].mode()

# Imprimir resultados
print(f"Media: {media}")
print(f"Mediana: {mediana}")
print(f"Moda: {moda.tolist()}")

Media: 4.983921754395742
Mediana: 5.0
Moda: [1]


In [155]:
# Comprobamos de nuevo las filas y columnas del MERGE de emociones y valoraciones
print(emocion_valoraciones.shape)

(35887, 4)


In [156]:
# Crear histograma para visualizar como se hacen las distribuciones
fig = px.histogram(
    emocion_valoraciones,
    x='valoracion',
    nbins=20,
    title='Distribución de Valoraciones',
    labels={'valoracion': 'Valoración'},
    color_discrete_sequence=['orange'],
    opacity=0.75
)

# Añadimos bordes a las barras
fig.update_traces(marker_line_width=1, marker_line_color='black')

# Personalizar diseño general
fig.update_layout(
    title={
        'text': 'Distribución de Valoraciones',
        'x': 0.5,
        'xanchor': 'center',
        'font': dict(size=22, family='Arial Black')
    },
    xaxis_title='Valoración',
    yaxis_title='Frecuencia',
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(255,255,255,1)',
    bargap=0.1,
    font=dict(size=14),
)

fig.show()

Interpretacion del histograma: Las frecuencias están muy parejas entre las diferentes valoraciones.Esto sugiere que las valoraciones están distribuidas de forma bastante homogénea, sin tendencia clara hacia lo positivo o negativo. La barra de la valoración "1" parece un poco más alta que las demás, coincide con lo que vimos de la moda, que es el valor que mas se repite quiza por personas que no quieren poner un 0 :) Haremos seguidamente un boxplot para ver la dispersión

In [157]:
# Gráfica de distibución - BOXPLOT
fig = px.box(
    emocion_valoraciones,
    y='valoracion',
    title='Boxplot de Valoraciones',
    points='all',  # para ver los puntos individuales
    color_discrete_sequence=['teal']
)

fig.update_layout(
    yaxis_title='Valoración',
    title_x=0.5
)

fig.show()

Interpretacion: Segun la caja central la mayoria de nuestras valoraciones va desde aproximadamente 2 hasta 8. Nuestra mediana es 5, lo que confirma que los datos están bastante centrados. Tiene sentido ademas que no haya outliers ya que nuestra escala esta cerrada de 0 -10.

#🌏 EDA PROCEDENCIA

In [160]:
# Comprobación de valores únicos - 34 en TOTAL: CORRECTO
print(df_id_visitante['procedencia'].unique())

['filipinas' 'españa' 'puerto rico' 'paraguay' 'argentina'
 'república dominicana' 'jamaica' 'ecuador' 'haití' 'chile' 'brasil'
 'canadá' 'honduras' 'alemania' 'el salvador' 'colombia' 'india' 'panamá'
 'perú' 'guinea-bissau' 'china' 'venezuela' 'italia' 'guinea ecuatorial'
 'cuba' 'uruguay' 'bolivia' 'estados unidos' 'francia' 'costa rica'
 'guatemala' 'méxico' 'nicaragua' 'trinidad y tobago']


In [161]:
df_id_visitante['procedencia'].describe()

,procedencia
count,35887
unique,34
top,españa
freq,8948


In [162]:
df_id_visitante['procedencia'].value_counts()

,count
procedencia,
españa,8948
el salvador,838
cuba,834
guinea-bissau,834
costa rica,829
argentina,828
jamaica,827
uruguay,824
chile,824


In [163]:
# Creo un diccionario para sustituír el resto de nombres para que aparezcan representados en la gráfica de Plotly
reemplazo_países = {
    'españa': 'Spain',
    'elemania': 'Germany',
    'francia': 'France',
    'brasil': 'Brazil',
    'canadá': 'Canada',
    'haití': 'Haiti',
    'estados unidos': 'United States',
    'filipinas': 'Philippines',
    'italia': 'Italy',
    'méxico': 'Mexico',
    'panamá': 'Panama',
    'perú': 'Peru',
    'república dominicana': 'Dominican Republic'
}

In [168]:
# Intento de gráfico con la distribución de las procedencias
procedencia_eda = df_id_visitante[['procedencia', 'id_visitante']].copy()

# Traducir si están en español
procedencia_eda['procedencia'] = procedencia_eda['procedencia'].replace(reemplazo_países)

# Asegurarse de contar 1 vez por visitante
df_mapa = procedencia_eda['procedencia'].value_counts().reset_index()
df_mapa.columns = ['procedencia', 'visitantes']
fig = px.choropleth(
    df_mapa,
    locations='procedencia',
    locationmode='country names',
    color='visitantes',
    color_continuous_scale='YlOrRd',
    title='Distribución de Visitantes por Procedencia',
    labels={'visitantes': 'Nº de Visitantes'}
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth'),
    title_x=0.5
)

fig.show()

In [169]:
# Creamos el listado del top 10 países por procedencia para usarlo posteriormente en gráfica
top_10_paises = df_mapa.sort_values(by='visitantes', ascending=False)
top_10_paises.head(10)

,procedencia,visitantes
0,Spain,8948
1,el salvador,838
2,cuba,834
3,guinea-bissau,834
4,costa rica,829
5,argentina,828
6,jamaica,827
7,uruguay,824
8,chile,824
9,ecuador,823


In [170]:
# Hacemos una representación del top 10
fig = px.bar(
    top_10_paises.head(10),
    x='visitantes',
    y='procedencia',
    orientation='h',
    text='visitantes',
    title='Top 10 Países por Número de Visitantes',
    labels={'visitantes': 'Visitantes', 'procedencia': 'País'},
    color='procedencia'
)

fig.update_traces(textposition='outside')
fig.update_layout(yaxis=dict(autorange='reversed'), showlegend=False)
fig.show()

# 🎢 EDA ATRACCIÓN

In [171]:
# Buscamos la cantidad de nulos dentro de nuestro DF
print("\nNulos por columna:\n", df_atracciones.isnull().sum())
# podemos observar que en la columna "atracción " hay 1134 nulos en una columna que es importante para el desarrollo de nuestro análisis


Nulos por columna:
 t_id                             0
atraccion                        0
comienzo_atraccion               0
tiempo_de_espera                 0
id_visitante                     0
comienzo_atraccion_fecha_hora    0
dtype: int64


In [172]:
# Calcular el número de filas con y sin nulos en la columna atraccion
nulos_tiempo = df_atracciones['comienzo_atraccion_fecha_hora'].isnull().sum()
no_nulos = len(df_atracciones) - nulos_tiempo

# Crear DataFrame para el gráfico
df_nulos_tiempo = pd.DataFrame({
    'Estado': ['Datos Válidos', 'Datos Nulos'],
    'Cantidad': [no_nulos, nulos]
})

# Calcular porcentajes para el texto
df_nulos_tiempo['Porcentaje'] = (df_nulos_tiempo['Cantidad'] / len(df_atracciones) * 100).round(2)

# Crear gráfico de barras
fig5 = px.bar(df_nulos, x='Estado', y='Cantidad',
             title='Porcentaje de Datos Válidos vs. Nulos en la Columna Atracción Fecha Hora',
             labels={'Cantidad': 'Número de Filas', 'Estado': 'Estado de los Datos'},
             color='Estado',
             text='Porcentaje',
             color_discrete_map={'Datos Válidos': '#00CC96', 'Datos Nulos': '#EF553B'})

# Personalizar el gráfico
fig5.update_traces(texttemplate='%{text:.2f}%', textposition='auto')
fig5.update_layout(yaxis_title='Número de Filas',
                  xaxis_title='Estado de los Datos',
                  showlegend=True,
                  bargap=0.3)

# Mostrar el gráfico
fig5.show()

In [173]:
# Contar el número de atracciones únicas
num_atracciones = df_atracciones['atraccion'].nunique()

# Crear DataFrame para el gráfico
df_grafico = pd.DataFrame({
    'Categoría': ['Atracciones Únicas'],
    'Cantidad': [num_atracciones]
})

# Crear gráfico de barras
fig = px.bar(df_grafico, x='Categoría', y='Cantidad',
             title='Cantidad de Atracciones Únicas en el Parque',
             labels={'Cantidad': 'Número de Atracciones', 'Categoría': ''},
             color='Categoría',
             text='Cantidad',
             color_discrete_map={'Atracciones Únicas': '#00CC96'})

# Personalizar el gráfico
fig.update_traces(texttemplate='%{text}', textposition='auto')
fig.update_layout(yaxis_title='Número de Atracciones',
                  xaxis_title='',
                  showlegend=False,
                  bargap=0.2)

# Mostrar el gráfico
fig.show()

In [174]:
# Filtrar filas con atraccion no nula
df_validas = df_atracciones[['atraccion']]

# Calcular los porcentajes de visitas por atracción
atracciones_counts = df_validas['atraccion'].value_counts(normalize=True) * 100
df_porcentajes = atracciones_counts.reset_index()
df_porcentajes.columns = ['Atracción', 'Porcentaje']

# Crear gráfico de barras
fig = px.bar(df_porcentajes, x='Atracción', y='Porcentaje',
             title='Porcentaje de Visitas por Atracción en el Parque',
             labels={'Porcentaje': 'Porcentaje de Visitas (%)', 'Atracción': 'Atracción'},
             color='Atracción',
             text='Porcentaje')

# Personalizar el gráfico
fig.update_traces(texttemplate='%{text:.1f}%', textposition='auto')
fig.update_layout(yaxis_title='Porcentaje de Visitas (%)',
                  xaxis_title='Atracción',
                  showlegend=False,
                  bargap=0.2)

# Mostrar el gráfico
fig.show()

In [178]:
visitantes_por_atraccion = df_id_visitante.groupby('atraccion')['id_visitante'].nunique().reset_index()
visitantes_por_atraccion.columns = ['atraccion', 'n_visitantes']
fig = px.bar(
    visitantes_por_atraccion,
    x='atraccion',
    y='n_visitantes',
    title='Número de Visitantes Únicos por Atracción',
    labels={'atraccion': 'Atracción', 'n_visitantes': 'Nº de Visitantes'},
    text='n_visitantes'
)


fig.show()

#🧭 EDA DURACIÓN

In [179]:
#Tendemos a ver que los primeros visitantes son los que más imágenes tienen
df1 = df_id_visitante['id_visitante'].value_counts().reset_index()

fig = px.scatter(df1,
             x='id_visitante',
             y='count',
             title='Imágenes por visitante',
             color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(title_x=0.5)

fig.show()

Como la duración ahora es un float (por la media), lo ideal es agrupar por duración redondeada (para que no tengas infinitos valores únicos) para poder visualizar los visitantes unicos por duracion media en una grafica de barras.

In [182]:
# Redondeamos la duración a enteros (minutos)
df_duracion['duracion_redondeada'] = df_duracion['duracion'].round(0).astype(int)

# Agrupar y preparar DataFrame para Plotly
duraciones_por_visitante = df_duracion['duracion_redondeada'].value_counts().sort_index().reset_index()
duraciones_por_visitante.columns = ['duracion_redondeada', 'visitantes']  # renombramos

fig = px.bar(
    duraciones_por_visitante,
    x='duracion_redondeada',
    y='visitantes',
    color='visitantes',
    color_continuous_scale='Turbo',  # Otras: 'Viridis', 'Blues', 'Inferno', 'Turbo'
    labels={'duracion': 'Duración (min)', 'visitantes': 'Visitantes'},
    title='Número de visitantes por duración media (minutos)'
)

fig.update_layout(title_x=0.5)
fig.show()

#🎫 EDA TICKET

Vamos a estudiar el df:
- Veremos en qué medida se venden los tipos de entrada.
- La diferencia de precios entre unas entradas y otras

In [183]:
#Observamos que todos los tipos entradas se venden por igual:
entradas = df_ticket['tipo_entrada'].value_counts().reset_index()

fig = px.pie(entradas,
             names='tipo_entrada',
             values='count',
             title='Porcentaje de tipos de entradas',
             color_discrete_sequence=px.colors.qualitative.Set1)

fig.update_layout(title_x=0.5)

fig.show()

In [184]:
#Observamos que todos los tipos entradas tiene precios similares:
entradas_2 = df_ticket.groupby('tipo_entrada')['coste'].mean().reset_index()

fig = px.bar(entradas_2,
             x='tipo_entrada',
             y='coste',
             title='Media de precio por entrada')

fig.update_layout(title_x=0.5)

fig.update_layout(yaxis_title='Precio entrada',
                  xaxis_title='Tipo de entrada',
                  showlegend=False,
                  title_x=0.5)

fig.show()

Aquí comprobamos que los precios de las entradas son iguales para todos los tipos de pases que hay. Debemos considerarlo un error, pero como no tenemos datos suficientes para cambiarlo, lo dejaremos como está.

In [185]:
#veremos ahora la distribucion de entradas despues de la correccion del fast pass

entradas_por_tipo = df_id_visitante.groupby('tipo_entrada')['id_visitante'].nunique().reset_index()
entradas_por_tipo.columns = ['tipo_entrada', 'n_visitas']
fig = px.pie(
    entradas_por_tipo,
    names='tipo_entrada',
    values='n_visitas',
    title='Distribución de Visitas por Tipo de Entrada'
)

fig.update_layout(title_x=0.5)
fig.show()

In [187]:
#vamos a ver distribucion de costes por tipo de entrada
coste_medio_por_tipo = df_id_visitante.groupby('tipo_entrada')['coste'].mean().reset_index()
coste_medio_por_tipo.columns = ['tipo_entrada', 'coste_medio']
coste_medio_por_tipo['coste_medio'] = coste_medio_por_tipo['coste_medio'].round(2)

fig = px.bar(
    coste_medio_por_tipo,
    x='tipo_entrada',
    y='coste_medio',
    title='Coste Medio por Tipo de Entrada',
    labels={'tipo_entrada': 'Tipo de Entrada', 'coste_medio': 'Coste Medio (€)'},
    text='coste_medio'
)

fig.update_layout(title_x=0.5)
fig.show()

# CREAR LOS DF PARA SQL A PARTIR DE DF_ID_VISITANTE

Hemos hecho la tabla relacional en SQL y necesitamos 4 tablas : emociones_valoraciones, procedencia, duracion y atracciones_ticket

In [188]:
df_id_visitante.columns

Index(['t_id', 'procedencia', 'tipo_entrada', 'coste', 'antelacion_de_compra',
       'atraccion', 'comienzo_atraccion', 'tiempo_de_espera',
       'comienzo_atraccion_fecha_hora', 'duracion', 'dia_visita',
       'id_visitante', 'fecha', 'hora', 'duracion_redondeada'],
      dtype='object')

In [193]:
duracion=df_id_visitante[['t_id', 'id_visitante', 'duracion']].copy()
duracion.head()
duracion.dtypes

,0
t_id,object
id_visitante,int64
duracion,int64


In [197]:
procedencia=df_id_visitante[['t_id', 'id_visitante', 'procedencia']].copy()
procedencia.head()
procedencia.dtypes

,0
t_id,object
id_visitante,int64
procedencia,object


In [199]:
ticket_atracciones=df_id_visitante[['t_id','id_visitante', 'atraccion', 'comienzo_atraccion_fecha_hora',
                                    'tiempo_de_espera','tipo_entrada', 'coste', 'antelacion_de_compra']].copy()
ticket_atracciones.head()



,t_id,id_visitante,atraccion,comienzo_atraccion_fecha_hora,tiempo_de_espera,tipo_entrada,coste,antelacion_de_compra
0,Training_10118481.jpg,16656,montaña rusa de la luna,2022-09-13 10:00:00,5,pase anual,14.00,0
1,Training_15361067.jpg,12164,mansión embrujada,2022-09-08 19:00:00,11,entrada familiar,20.90,2
2,Training_16232328.jpg,28210,simulador espacial 3d,2022-09-17 07:00:00,19,entrada infantil,12.64,218
3,Training_27705571.jpg,26769,fiesta de los dulces,2022-09-27 08:00:00,9,paquete vip,18.49,39
4,Training_63766171.jpg,794,gran caída libre,2022-09-26 20:00:00,17,entrada familiar,18.23,135


#📊CREACION DE LOS CSV DE DATOS

In [200]:
#📊CREACION DE LOS CSV DE DATOS

#creacion de los CSV

#Vamos a crear una función que reciba un diccionario con los nombres de archivo y
#los DataFrames correspondientes, y los guarde todos como CSV con los parámetros que estás usando (UTF-8, sep=";", index=False).
#Parámetros:
   # - dataframes_dict: dict -> Diccionario con nombre_de_archivo (sin .csv) como clave y DataFrame como valor.
    #- ruta_base: str -> Carpeta donde se guardarán los archivos. Por defecto: "/content/"


def guardar_csv(dataframes_dict, ruta_base="/content/"):

    for nombre_archivo, df in dataframes_dict.items():
        ruta_completa = f"{ruta_base}{nombre_archivo}.csv"
        sep = "," if nombre_archivo != "id_visitante_ticket_limpio" else ","
        df.to_csv(ruta_completa, index=False, encoding="utf-8", sep=sep) # cambiar a Latin si se necesitan los caracteres especiales
        print(f"Archivo guardado: {ruta_completa}")

dataframes = {
    "procedencia": procedencia,
    "duracion": duracion,
    "ticket_atracciones": ticket_atracciones,
    "valoraciones_emociones": emocion_valoraciones}


guardar_csv(dataframes)


Archivo guardado: /content/procedencia.csv
Archivo guardado: /content/duracion.csv
Archivo guardado: /content/ticket_atracciones.csv
Archivo guardado: /content/valoraciones_emociones.csv
